# Bokeh

In [1]:
import numpy as np
import pandas as pd

In [2]:
from datetime import datetime

from bokeh.io import show, output_notebook, output_file, curdoc
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.models import RangeTool, CrosshairTool, BoxSelectTool, HoverTool, CustomJS
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import figure
from bokeh.palettes import Category20
from bokeh.models.widgets import PreText, DataTable, TableColumn, DateFormatter, HTMLTemplateFormatter, TextAreaInput
from bokeh.sampledata.stocks import AAPL

---

The basic steps to creating plots with the bokeh.plotting interface are:

- **Prepare some data**
    - In this case plain python lists, but could also be NumPy arrays or Pandas series.
- **Tell Bokeh where to generate output**
    - In this case using `output_file()`, with the filename "lines.html".
    - Another option is `output_notebook()` for use in Jupyter notebooks.
- **Call `figure()`**
    - This creates a plot with typical default options and easy customization of title, tools, and axes labels.
- **Add renderers**
    - In this case, we use `line()` for our data, specifying visual customizations like colors, legends and widths.
- **Ask Bokeh to `show()` or `save()` the results**
    - These functions save the plot to an HTML file and optionally display it in a browser.

---

Useful Links:

- http://www.colors.commutercreative.com/

---

## Sample Data

In [3]:
np.random.seed(0)

df = pd.DataFrame(AAPL)[['date', 'adj_close']]
df.date = pd.to_datetime(df.date)
df.adj_close = df.adj_close.rolling(20).mean()
df = df.loc[(df.date >= '2001-01-01') & (df.date < '2004-01-01'), :]

df.rename(columns={'adj_close':'px'}, inplace=True)

df['bid'] = df.px - 0.5
df['ask'] = df.px + 0.5

df['rw1'] = np.cumsum(np.random.normal(size=df.shape[0]))
df['rw2'] = np.cumsum(np.random.normal(size=df.shape[0]))

trade = pd.DataFrame(df.date.sample(100))

trade['size'] = np.random.randint(10, 20, size=100)
trade['ex'] = np.random.choice(['A','B','C'], size=100)

trade = trade.merge(df.px, how='left', left_index=True, right_index=True).sort_index()

trade['alert_msg'] = ['This is a very very very long alert message ' + str(i) for i in range(trade.shape[0])]

In [4]:
df.shape

(752, 6)

In [5]:
df.head()

,date,px,bid,ask,rw1,rw2
212,2001-01-02,7.2265,6.7265,7.7265,1.764052,0.448195
213,2001-01-03,7.2185,6.7185,7.7185,2.164210,2.144377
214,2001-01-04,7.2200,6.7200,7.7200,3.142948,2.129519
215,2001-01-05,7.2700,6.7700,7.7700,5.383841,2.950925
216,2001-01-08,7.3245,6.8245,7.8245,7.251399,3.621496


In [6]:
trade.head()

,date,size,ex,px,alert_msg
223,2001-01-18,17,A,7.7665,This is a very very very long alert message 0
230,2001-01-29,19,C,8.7200,This is a very very very long alert message 1
231,2001-01-30,18,B,8.8870,This is a very very very long alert message 2
245,2001-02-20,11,C,9.8450,This is a very very very long alert message 3
251,2001-02-28,15,C,9.5935,This is a very very very long alert message 4


---

## Visualization

In [7]:
def add_vlinked_crosshairs(fig1, fig2):
    js_move = '''if(cb_obj.x >= fig.x_range.start && cb_obj.x <= fig.x_range.end && cb_obj.y >= fig.y_range.start && cb_obj.y <= fig.y_range.end)
                    { cross.spans.height.computed_location = cb_obj.sx }
                 else 
                    { cross.spans.height.computed_location = null }'''
    js_leave = 'cross.spans.height.computed_location = null'

    cross1 = CrosshairTool()
    cross2 = CrosshairTool()
    fig1.add_tools(cross1)
    fig2.add_tools(cross2)
    args = {'cross': cross2, 'fig': fig1}
    fig1.js_on_event('mousemove', CustomJS(args = args, code = js_move))
    fig1.js_on_event('mouseleave', CustomJS(args = args, code = js_leave))
    args = {'cross': cross1, 'fig': fig2}
    fig2.js_on_event('mousemove', CustomJS(args = args, code = js_move))
    fig2.js_on_event('mouseleave', CustomJS(args = args, code = js_leave))

In [8]:
source = ColumnDataSource(df)
tradeSource = ColumnDataSource(trade)

output_file('sample.html', mode='inline')

############################################################

plot = figure(plot_height=400, plot_width=800, tools="pan,wheel_zoom,tap,reset", toolbar_location='right',
              x_axis_type="datetime", x_axis_location=None,
              background_fill_color="#efefef", x_range=(df.date.iloc[10], df.date.iloc[110]))

plot.line('date', 'bid', color='DodgerBlue', line_width=2, source=source)
plot.line('date', 'ask', color='Crimson', line_width=2, source=source)

for i, mkt in enumerate(trade.ex.unique()):
    viewMkt = CDSView(source=tradeSource, filters=[GroupFilter(column_name='ex', group=mkt)])
    plot.scatter(x='date', y='px', size='size', color=Category20[20][i], 
                 alpha=0.5, source=tradeSource, view=viewMkt, legend=mkt)

plot.add_tools(BoxSelectTool(dimensions='width')) # to clear selection, press ESC

data = dict(
    date = [],
    px   = [],
    size = [],
    alert_msg = []
)
columns = [
    TableColumn(field='date', title='Date', width=100, formatter=DateFormatter()),
    TableColumn(field='px',   title='Price', width=300),
    TableColumn(field='size', title='Size', width=10),
    TableColumn(field='alert_msg', title='Alert')
]
selectedTradeSource = ColumnDataSource(data)
fillTable = DataTable(source=selectedTradeSource, columns=columns, editable=True, width=500)

tradeSource.selected.js_on_change('indices', CustomJS(args=dict(s1=tradeSource, s2=selectedTradeSource), code='''
    var inds = cb_obj.indices;
    var d1 = s1.data;
    var d2 = s2.data;
    d2['date'] = []
    d2['px']   = []
    d2['size'] = []
    d2['alert_msg'] = []
    for (var i = 0; i < inds.length; i++) {
        d2['date'].push(d1['date'][inds[i]])
        d2['px'].push(d1['px'][inds[i]])
        d2['size'].push(d1['size'][inds[i]])
        d2['alert_msg'].push(d1['alert_msg'][inds[i]])
    }
    s2.change.emit();
'''
))

plot.legend.location = 'bottom_left'
plot.legend.orientation = 'horizontal'
plot.legend.click_policy = 'hide'

############################################################

plot2 = figure(plot_height=130, plot_width=800, tools='xpan', toolbar_location=None,
               background_fill_color='#efefef', x_range=plot.x_range)

plot2.line('date', 'rw1', color='DodgerBlue', line_width=2, source=source)

hover = HoverTool(
    tooltips=[
        ('Date', '@date{%F}'),
        ('Price', '@px')
    ],
    formatters={
        'date':'datetime'
    },
    mode='vline'
)

plot2.add_tools(hover)

add_vlinked_crosshairs(plot, plot2)

############################################################

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=plot.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'rw2', color='DodgerBlue', line_width=2, source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

############################################################

#layout = gridplot([[plot2], [plot], [select]])
main = column(plot2, plot, select)
message = column(fillTable)
layout = row(main, message)
show(layout)